In [1]:
import os
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import re
import requests

In [2]:
os.listdir('html')

['4.html',
 '2.html',
 '3.html',
 '8.html',
 '11.html',
 '9.html',
 '7.html',
 '6.html',
 '5.html',
 '10.html']

In [3]:
fnames = os.listdir("html")

def html_to_df(fname):
    with open(os.path.join("html", fname)) as f:
        html = f.read()
    
    doc = BeautifulSoup(html, 'html.parser')
    
    for i in doc:
        address = doc.find_all (class_= 'list-card-addr')
        price = list(doc.find_all (class_='list-card-price'))
        beds = list(doc.find_all("ul", class_="list-card-details"))
        details = doc.find_all ('div', {'class': 'list-card-details'})
        home_type = doc.find_all ('div', {'class': 'list-card-footer'})
        last_updated = doc.find_all ('div', {'class': 'list-card-top'})
        brokerage = list(doc.find_all(class_= 'list-card-brokerage list-card-img-overlay',text=True))
    
    zillow = pd.DataFrame()
    
    zillow['address'] = address
    zillow['prices'] = price
    zillow['beds'] = beds
    
    return zillow

In [4]:
fnames = os.listdir("html")
dfs = list()
for fname in fnames:
    dfs.append(html_to_df(fname))
    
all_df = pd.concat(dfs).reset_index(drop=True)

In [5]:
#convert columns to str
all_df['prices'] = all_df['prices'].astype('str')
all_df['address'] = all_df['address'].astype('str')
all_df['beds'] = all_df['beds'].astype('str')


In [6]:
#remove html tags
all_df['prices'] = all_df['prices'].replace('<div class="list-card-price">', ' ', regex=True)
all_df['address'] = all_df['address'].replace('<address class="list-card-addr">', ' ', regex=True)
all_df['prices'] = all_df['prices'].replace('</div>', ' ', regex=True)
all_df['address'] = all_df['address'].replace('</address>', ' ', regex=True)
all_df['prices'] = all_df['prices'].str.replace(r'\D', '')

In [7]:
#remove html tags from beds column
all_df['beds'] = all_df['beds'].replace('<ul class="list-card-details"><li class="">', ' ', regex=True)
all_df['beds'] = all_df['beds'].replace('<abbr class="list-card-label"> <!-- -->bds</abbr></li><li class="">', ', ', regex=True)
all_df['beds'] = all_df['beds'].replace('<abbr class="list-card-label"> <!-- -->bd</abbr></li><li class="">', ', ', regex=True)
all_df['beds'] = all_df['beds'].replace('<abbr class="list-card-label"> <!-- -->ba</abbr></li><li class="">', ', ', regex=True)
all_df['beds'] = all_df['beds'].replace('<abbr class="list-card-label"> <!-- -->sqft</abbr></li><li class="list-card-statusText">-', ', ', regex=True)
all_df['beds'] = all_df['beds'].replace('<abbr class="list-card-label"> <!-- -->sqft lot</abbr></li><li class="list-card-statusText">-', ', ', regex=True)
all_df['beds'] = all_df['beds'].replace('<abbr class="list-card-label"> <!-- -->acres lot</abbr></li><li class="list-card-statusText">-', ', ', regex=True)
all_df['beds'] = all_df['beds'].replace('</li></ul>', ' ', regex=True)

all_df

,address,prices,beds
0,"1821 Schlimgen Ave, Madison, WI 53704",189900,"2, 1, 632, House for sale"
1,"10 Kings Mill Cir UNIT 114, Madison, WI 53718",175000,"2, 2, 1,216, Condo for sale"
2,"4663 Treichel St APT 101, Madison, WI 53718",168900,"2, 1, 981, Condo for sale"
3,"7478 East Pass, Madison, WI 53719",259900,"2, 3, 1,551, Condo for sale"
4,"3914 Cosgrove Dr, Madison, WI 53719",359900,"3, 3, 2,277, House for sale"
...,...,...,...
395,"Lot 154 Plan, Woodstone",490000,"3, 2, 1,648, New construction"
396,"The Rousseau Plan, Village at Autumn Lake",424600,"3, 3, 1,905, New construction"
397,"1801 Opus Ln #204, Verona, WI 53593",432900,"2, 2, 1,780, Condo for sale"
398,"1801 Opus Ln #104, Verona, WI 53593",424900,"2, 2, 1,698, Condo for sale"


In [8]:
#split beds column into beds, bath, sq_feet, and type

part_df = pd.DataFrame(all_df.beds.str.split(', ').tolist(),
                                 columns = ['beds','baths','sq_feet', 'type'])

In [9]:
all_df.pop('beds')

0                     2, 1, 632,  House for sale 
1                   2, 2, 1,216,  Condo for sale 
2                     2, 1, 981,  Condo for sale 
3                   2, 3, 1,551,  Condo for sale 
4                   3, 3, 2,277,  House for sale 
                          ...                    
395               3, 2, 1,648,  New construction 
396               3, 3, 1,905,  New construction 
397                 2, 2, 1,780,  Condo for sale 
398                 2, 2, 1,698,  Condo for sale 
399     5, 4, 4,635,  Multi-family home for sale 
Name: beds, Length: 400, dtype: object

In [10]:
# merge
all_df = all_df.merge(part_df, left_index=True, right_index=True, how='inner')


In [11]:
# adjust columns
all_df.replace(',','', regex=True, inplace=True)

In [12]:
pd.set_option('max_rows', None)

In [13]:
# delete rows containing none or new construction
all_df = all_df.replace(to_replace='None', value=np.nan).dropna()
all_df = all_df[all_df["type"].str.contains("New construction")==False]

In [14]:
# replace missing values with 0
all_df['beds'] = all_df['beds'].replace('--', '0', regex=True)
all_df['baths'] = all_df['baths'].replace('--', '0', regex=True)
all_df['sq_feet'] = all_df['sq_feet'].replace('--', '0', regex=True)

# convert columns to float
all_df['prices'] = all_df['prices'].astype('int')
all_df['beds'] = all_df['beds'].astype('int')
all_df['baths'] = all_df['baths'].astype('int')
all_df['sq_feet'] = all_df['sq_feet'].astype('int')

In [15]:
# shorten type
all_df['type'] = all_df['type'].str.slice(0, -9)
all_df['type'] = all_df['type'].str.strip()

In [16]:
# add zip code col
all_df['zip'] = all_df['address'].str.slice(-6).astype(int)

In [17]:
# delete all rows with missing data / 0.0 row
all_df = res = all_df[~all_df.eq(0.0).any(1)]

In [18]:
#prices in $1,000
all_df['prices'] = (all_df['prices'] / 1000).round()
#sq_feet in 1,000s
all_df['sq_feet'] = (all_df['sq_feet'] / 1000).round(1) 

In [19]:
# only want types with a max of 4 beds, 4 baths, 4,000 sq_feet
all_df = all_df[all_df['beds'] <= 4.0]
all_df = all_df[all_df['baths'] <= 4.0]
all_df = all_df[all_df['sq_feet'] <= 3.5]

In [20]:
pd.set_option('max_rows', None)
all_df

,address,prices,beds,baths,sq_feet,type,zip
0,1821 Schlimgen Ave Madison WI 53704,190.0,2,1,0.6,House,53704
1,10 Kings Mill Cir UNIT 114 Madison WI 53718,175.0,2,2,1.2,Condo,53718
2,4663 Treichel St APT 101 Madison WI 53718,169.0,2,1,1.0,Condo,53718
3,7478 East Pass Madison WI 53719,260.0,2,3,1.6,Condo,53719
4,3914 Cosgrove Dr Madison WI 53719,360.0,3,3,2.3,House,53719
5,637 Chatham Ter Madison WI 53711,565.0,4,4,2.4,House,53711
7,606 S Baldwin St Madison WI 53703,350.0,2,1,1.0,House,53703
8,4410 Wakefield St Madison WI 53711,370.0,3,1,1.4,House,53711
9,69 White Oaks Ln Madison WI 53711,495.0,4,3,3.4,House,53711
10,626 Frederick Ln Madison WI 53711,365.0,2,1,1.2,House,53711


In [21]:
all_df.to_csv('zillow.csv', index=False)